In [ ]:
!mkdir models

In [ ]:
!git clone https://huggingface.co/BrandonFors/effnetv2_s_plant_disease
!cp ./effnetv2_s_plant_disease/effnetv2_s_plant_disease.pth ./models/
!rm -r ./effnetv2_s_plant_disease

Cloning into 'effnetv2_s_plant_disease'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 55 (delta 9), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (55/55), 11.66 KiB | 746.00 KiB/s, done.
Filtering content: 100% (28/28), 233.83 MiB | 56.58 MiB/s, done.


In [ ]:
# get personal MLOps Utils
from pathlib import Path
import requests
import os
import zipfile

utils_path = Path("utils/")
utils_path.mkdir(exist_ok=True, parents=True)

with open(utils_path / "utils.zip", "wb") as f:
  request = requests.get("https://github.com/BrandonFors/MLOps_Utils/raw/main/utils.zip")
  print("Downloading utils")
  f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(utils_path / "utils.zip", "r") as zip_ref:
  print("Unzipping utils")
  zip_ref.extractall(utils_path)

# Remove .zip file
os.remove(utils_path / "utils.zip")

Unzipping utils


In [ ]:
import torch
import torchvision

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Make the model compatable with hugging face

In [1]:
# Use this when creating files to be uploaded to hub
# %%writefile model_upload/configuration.py
import transformers
from transformers import PretrainedConfig
# create a config class
class EffNetPlantDiseaseConfig(PretrainedConfig):
  # tells hugging face the model family type
  # could be used to register class with AutoConfig
  model_type = "effnetv2_s_plant_disease"

  def __init__(self,
               num_classes=38,
               image_size=384,
               dropout_rate = 0.2,
               class_names = None,
               **kwargs):
    super().__init__(**kwargs)
    # assign inputs
    self.num_classes = num_classes
    self.image_size = image_size # unused but will keep just in case
    self.dropout_rate = dropout_rate
    # create class dictionaries from the label (image class) names
    if class_names:
      self.id2label = {str(i): name for i, name in enumerate(class_names)}
      self.label2id = {name: str(i) for i, name in enumerate(class_names)}
    else:
      self.id2label = {str(i): f"class_{i}" for i in range(num_classes)}
      self.label2id = {f"class_{i}": str(i) for i in range(num_classes)}





In [2]:
# use this when creating files to upload to the hub
# %%writefile model_upload/modeling.py
import torch
import torchvision
from torch import nn
from transformers import PreTrainedModel
# uncomment when writing file
# from configuration import EffNetPlantDiseaseConfig

# create model class
class EffNetPlantDiseaseClassification(PreTrainedModel):
  config_class = EffNetPlantDiseaseConfig
  def __init__(self, config):
    super().__init__(config)
    # get the model architecture from torchvision
    self.model = torchvision.models.efficientnet_v2_s()
    # modify the classifier head according to the config
    self.model.classifier = nn.Sequential(
        nn.Dropout(p=config.dropout_rate, inplace=True),
        nn.Linear(in_features=self.model.classifier[-1].in_features,
                  out_features=config.num_classes)
      )
    self.num_classes = config.num_classes
    self.loss_fn =  nn.CrossEntropyLoss()
  # define forward method to be similar to hugging face model standards
  def forward(self, image, label=None):
    logits = self.model(image)
    loss = None
    if label is not None:
      loss = self.loss_fn(logits, label)

    return {"logits":logits,
            "loss": loss}


In [ ]:
class_names = ['Apple___Apple_scab',
  'Apple___Black_rot',
  'Apple___Cedar_apple_rust',
  'Apple___healthy',
  'Blueberry___healthy',
  'Cherry_(including_sour)___Powdery_mildew',
  'Cherry_(including_sour)___healthy',
  'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
  'Corn_(maize)___Common_rust_',
  'Corn_(maize)___Northern_Leaf_Blight',
  'Corn_(maize)___healthy',
  'Grape___Black_rot',
  'Grape___Esca_(Black_Measles)',
  'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
  'Grape___healthy',
  'Orange___Haunglongbing_(Citrus_greening)',
  'Peach___Bacterial_spot',
  'Peach___healthy',
  'Pepper,_bell___Bacterial_spot',
  'Pepper,_bell___healthy',
  'Potato___Early_blight',
  'Potato___Late_blight',
  'Potato___healthy',
  'Raspberry___healthy',
  'Soybean___healthy',
  'Squash___Powdery_mildew',
  'Strawberry___Leaf_scorch',
  'Strawberry___healthy',
  'Tomato___Bacterial_spot',
  'Tomato___Early_blight',
  'Tomato___Late_blight',
  'Tomato___Leaf_Mold',
  'Tomato___Septoria_leaf_spot',
  'Tomato___Spider_mites Two-spotted_spider_mite',
  'Tomato___Target_Spot',
  'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
  'Tomato___Tomato_mosaic_virus',
  'Tomato___healthy']

In [ ]:
# create a config
config = EffNetPlantDiseaseConfig(num_classes=len(class_names), class_names=class_names)

In [ ]:
test_dir = Path("test_dir/")
test_dir.mkdir(exist_ok=True, parents=True)
#save the config to be tested with loading
config.save_pretrained("./test_dir")

In [ ]:
# load the config and look at its contents
loaded_config = EffNetPlantDiseaseConfig.from_pretrained(test_dir)
loaded_config

EffNetPlantDiseaseConfig {
  "dropout_rate": 0.2,
  "id2label": {
    "0": "class_0",
    "1": "class_1",
    "10": "class_10",
    "11": "class_11",
    "12": "class_12",
    "13": "class_13",
    "14": "class_14",
    "15": "class_15",
    "16": "class_16",
    "17": "class_17",
    "18": "class_18",
    "19": "class_19",
    "2": "class_2",
    "20": "class_20",
    "21": "class_21",
    "22": "class_22",
    "23": "class_23",
    "24": "class_24",
    "25": "class_25",
    "26": "class_26",
    "27": "class_27",
    "28": "class_28",
    "29": "class_29",
    "3": "class_3",
    "30": "class_30",
    "31": "class_31",
    "32": "class_32",
    "33": "class_33",
    "34": "class_34",
    "35": "class_35",
    "36": "class_36",
    "37": "class_37",
    "4": "class_4",
    "5": "class_5",
    "6": "class_6",
    "7": "class_7",
    "8": "class_8",
    "9": "class_9"
  },
  "image_size": 384,
  "label2id": {
    "class_0": "0",
    "class_1": "1",
    "class_10": "10",
    "class_11":

In [ ]:
# create the model
model = EffNetPlantDiseaseClassification(config)
model

EffNetPlantDiseaseClassification(
  (model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            

In [ ]:
# get the state dict from the files
state_dict = torch.load("./models/effnetv2_s_plant_disease.pth", map_location=torch.device("cpu"))
state_dict

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# create a new state dict with model appended to each of the keys to work with the hugging face compatible model
new_state_dict = {}
for k, v in state_dict.items():
  k = f"model.{k}"
  new_state_dict[k] = v
new_state_dict

{'model.features.0.0.weight': tensor([[[[-5.4273e-01, -3.5956e-02,  6.3242e-01],
           [-1.0378e+00, -1.2940e-01,  1.2503e+00],
           [-1.0799e+00, -1.8302e-01,  1.1356e+00]],
 
          [[-8.5667e-01, -1.1255e-01,  8.6508e-01],
           [-1.4344e+00, -1.9008e-01,  1.6750e+00],
           [-1.4068e+00, -1.5620e-01,  1.6076e+00]],
 
          [[-2.8686e-01, -5.7373e-02,  3.3806e-01],
           [-6.4328e-01, -3.0316e-02,  9.3808e-01],
           [-7.2246e-01, -2.1713e-01,  7.5683e-01]]],
 
 
         [[[ 9.1505e-02,  8.0611e-02, -8.2176e-02],
           [ 1.7246e-01, -4.2963e-02, -1.4263e-01],
           [ 1.1775e-01, -3.8887e-02, -1.8789e-01]],
 
          [[-2.2585e-02, -3.3523e-01, -2.9263e-01],
           [-3.5476e-01, -7.0119e-01, -5.3781e-01],
           [-2.6268e-01, -4.9725e-01, -4.9300e-01]],
 
          [[-3.1894e-04,  2.7804e-01,  3.0494e-01],
           [ 1.6055e-01,  8.0333e-01,  8.4293e-01],
           [ 4.4523e-02,  6.7281e-01,  6.2647e-01]]],
 
 
         [[

In [ ]:
# load the new state dict
model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [ ]:
from pathlib import Path
upload_dir = Path("model_upload/")
upload_dir.mkdir(exist_ok=True, parents = True)
# save the config and model
config.save_pretrained(upload_dir)
model.save_pretrained(upload_dir)
# save the new state dict
pytorch_bin_pth = upload_dir / "pytorch_model.bin"
torch.save(new_state_dict, pytorch_bin_pth)
files = os.listdir(upload_dir)


In [ ]:
# create a requirements file
%%writefile model_upload/requirements.txt
torch
torchvision
transformers

Writing model_upload/requirements.txt


## Push model files to hugging face

In [ ]:
from huggingface_hub import login
login()

In [ ]:
!git config --global user.email ""
!git config --global user.name ""

In [ ]:
!git clone https://huggingface.co/BrandonFors/effnetv2_s_plant_disease
!cp -r model_upload/* ./effnetv2_s_plant_disease

Cloning into 'effnetv2_s_plant_disease'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 55 (delta 9), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (55/55), 11.66 KiB | 995.00 KiB/s, done.
Filtering content: 100% (28/28), 233.83 MiB | 48.77 MiB/s, done.


In [ ]:
%cd effnetv2_s_plant_disease/

/content/effnetv2_s_plant_disease


In [ ]:
!git add *

In [ ]:
!git commit -m "upload model files"

[main 79cd17f] upload model files
 3 files changed, 8 insertions(+), 7 deletions(-)


In [ ]:
!git push

Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 520 bytes | 520.00 KiB/s, done.
Total 5 (delta 4), reused 0 (delta 0), pack-reused 0
To https://huggingface.co/BrandonFors/effnetv2_s_plant_disease
   92d8793..79cd17f  main -> main


In [5]:
# grab the model and pretrained weights from the hugging face repo
test_model = EffNetPlantDiseaseClassification.from_pretrained("BrandonFors/effnetv2_s_plant_disease").to(device)
test_model

EffNetPlantDiseaseClassification(
  (model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            

## Test the loaded model

* code is taken from the model evaluation notebook

In [6]:
# run a test
test_model(torch.rand(1,3,384,384).to(device))["logits"]

tensor([[0.6700, 0.8875, 0.7995, 0.9444, 1.3761, 0.7952, 1.7495, 1.5474, 1.4057,
         1.3514, 3.3264, 1.1229, 0.4266, 0.9397, 1.4925, 0.0755, 0.5435, 1.6952,
         0.0392, 1.1035, 0.6862, 0.5336, 2.1573, 1.5589, 1.7234, 1.7868, 0.7034,
         1.5034, 0.7135, 0.8532, 0.3161, 0.6849, 0.6243, 1.3669, 1.4217, 1.0773,
         1.8491, 1.1624]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [8]:
# install torchmetrics
try:
  import torchmetrics
except:
  !pip install torchmetrics
  import torchmetrics

In [ ]:
from datasets import load_dataset, ClassLabel
# load the test split of the dataset
test_dataset = load_dataset("BrandonFors/Plant-Diseases-PlantVillage-Dataset", split="test")

# Get the class names from the dataset and create int labels for the classes
class_names = test_dataset.features["label"].names
class_label = ClassLabel(names=class_names)

# apply the int class labels to the dataset
test_dataset = test_dataset.cast_column("label", class_label)

test_dataset, class_names

data/train-00000-of-00002.parquet:   0%|          | 0.00/321M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/362M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/170M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43456 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10849 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/10849 [00:00<?, ? examples/s]

(Dataset({
     features: ['image', 'label'],
     num_rows: 10849
 }),
 ['Apple___Apple_scab',
  'Apple___Black_rot',
  'Apple___Cedar_apple_rust',
  'Apple___healthy',
  'Blueberry___healthy',
  'Cherry_(including_sour)___Powdery_mildew',
  'Cherry_(including_sour)___healthy',
  'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
  'Corn_(maize)___Common_rust_',
  'Corn_(maize)___Northern_Leaf_Blight',
  'Corn_(maize)___healthy',
  'Grape___Black_rot',
  'Grape___Esca_(Black_Measles)',
  'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
  'Grape___healthy',
  'Orange___Haunglongbing_(Citrus_greening)',
  'Peach___Bacterial_spot',
  'Peach___healthy',
  'Pepper,_bell___Bacterial_spot',
  'Pepper,_bell___healthy',
  'Potato___Early_blight',
  'Potato___Late_blight',
  'Potato___healthy',
  'Raspberry___healthy',
  'Soybean___healthy',
  'Squash___Powdery_mildew',
  'Strawberry___Leaf_scorch',
  'Strawberry___healthy',
  'Tomato___Bacterial_spot',
  'Tomato___Early_blight',
  'Tomato___La

In [ ]:
# Get model weights from torchvision
import torchvision
### EffNetV2 - S
effnetv2_s_weights = torchvision.models.EfficientNet_V2_S_Weights.DEFAULT
effnetv2_s_auto_transforms = effnetv2_s_weights.transforms()

In [ ]:
# apply the train transform to train datasets and the base auto transform to test datasets
from utils.vision_utils import apply_transforms

### EffnetV2
effnetv2_s_test_dataset = test_dataset.with_transform(lambda x: apply_transforms(x, effnetv2_s_auto_transforms))

In [ ]:
import os
from torch.utils.data import DataLoader

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

### EffnetV2
effnetv2_s_test_dataloader = DataLoader(dataset=effnetv2_s_test_dataset,
                                           batch_size=BATCH_SIZE,
                                           num_workers=NUM_WORKERS)

In [ ]:
from typing import List, Dict
from timeit import default_timer as timer
from torchmetrics import F1Score, Accuracy
from tqdm.auto import tqdm

def evaluate_model(dataset,
                   dataloader,
                   model:nn.Module,
                   loss_fn: nn.Module,
                   class_names:List[str],
                   model_name:str,
                   device:torch.device = device):
  # Create a function that calculates the overall loss, accuracy, F1 score, avg time to predict, model size,

  loss_list = []
  acc_list = []
  f1_list = []
  time_list = []

  # get metrics
  f1 = F1Score(task="multiclass", num_classes=len(class_names), average="macro").to(device)
  acc = Accuracy(task="multiclass", num_classes=len(class_names)).to(device)

  model.to(device)

  # Loop through paths
  for step, batch in tqdm(enumerate(dataloader),total=len(dataloader) ):

    # move batch to device
    batch["image"], batch["label"] = batch["image"].to(device), batch["label"].to(device)

    # Prep the model for prediciton
    model.eval()
    with torch.inference_mode():
      # get the logits
      logits = model(batch["image"])["logits"]
      # get the loss
      loss = loss_fn(logits,batch["label"])
      # Get pred probabilities
      pred_probs = torch.softmax(logits, dim = 1)
      # Get the pred label
      pred_label = torch.argmax(pred_probs, dim=1)


      f1_score = f1(pred_label, batch["label"])
      accuracy = acc(pred_label, batch["label"])

      # Add data to appropriate list dictionary
      loss_list.append(loss.item())
      acc_list.append(accuracy.item())
      f1_list.append(f1_score.item())


  avg_loss = round(sum(loss_list)/len(loss_list),4)
  avg_acc = round(sum(acc_list)/len(acc_list),4)
  avg_f1 = round(sum(f1_list)/len(f1_list),4)

  # loop through 25 items from the dataset to get an average prediction time
  model.to(torch.device("cpu"))
  for idx in tqdm(range(25)):

    # Prep the model for prediciton
    model.eval()
    with torch.inference_mode():
      # Start the timer
      start_time = timer()
      # get the logits
      logits = model(dataset[idx]["image"].unsqueeze(dim=0))["logits"]
      # get the loss
      loss = loss_fn(logits,torch.tensor(dataset[idx]["label"]).unsqueeze(dim=0))
      # Get pred probabilities
      pred_probs = torch.softmax(logits, dim = 1)
      # Get the pred label
      pred_label = torch.argmax(pred_probs, dim=1)

      # stop the timer
      end_time = timer()
      # Calculate the total time
      total_time = end_time-start_time
      time_list.append(total_time)


  avg_time = round(sum(time_list)/len(time_list),4)

  stats_dict = {
      "loss":avg_loss,
      "accuracy":avg_acc,
      "f1_score":avg_f1,
      "prediction_time":avg_time,
      "model_name":model_name
  }

  return stats_dict

In [ ]:
from torch import nn

effnetv2_s_test_data = evaluate_model(dataset=effnetv2_s_test_dataset,
                                      dataloader=effnetv2_s_test_dataloader,
                                      model=test_model,
                                      loss_fn=nn.CrossEntropyLoss(),
                                      class_names=class_names,
                                      model_name="test_model",
                                      device=device)
effnetv2_s_test_data

  0%|          | 0/340 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

{'loss': 0.0144,
 'accuracy': 0.9963,
 'f1_score': 0.979,
 'prediction_time': 0.3451,
 'model_name': 'test_model'}

### Results
The model and weights have successfully been downloaded using the custom config and model classes created using the hugging face transformers library